Load CSV and combine to excel

In [ ]:
"""
STEP1: Group CSVs into Excel Files
----------------------------------------------------------------

This script processes per-channel CSV files exported from a calcium imaging experiment and
combines them into structured Excel files, one per image. It assumes that each image has
four associated channels (e.g., FB1 to FB4), stored in individual CSVs with a shared base name.

Functionality:
- Scans a folder for all `.csv` files.
- Groups files by image base name, expecting four channels per image.
- Sorts channel files to ensure consistent FP1 to FP4 order.
- Loads CSV data and writes each channel to a separate sheet in a single Excel file.

Assumptions:
- CSVs are named using the format "<base_name>_FBX.csv", where X ∈ {1,2,3,4}.
- All CSV files are located in the same directory (`file_path`).

Outputs:
- One Excel file per image: "<base_name>_analysis.xlsx"
  - Sheets: FB1, FB2, FB3, FB4 — raw per-channel intensity data.

Usage:
- Update `file_path` to point to the directory containing the CSV files.
- Run in a Python environment with `pandas` installed.
"""


import pandas as pd
import os
from collections import defaultdict

# Define the path to your CSV files
file_path = r"your\file\path"  # Replace with the actual path to your files

# Set the suffix for the output Excel file
excel_suffix = '_analysis'

# Dictionary to hold the grouped CSV files
grouped_files = defaultdict(list)

# Loop through the files in the folder
for file in os.listdir(file_path):
    if file.endswith(".csv"):
        # Split the file name into base and suffix (FB1, FB2, etc.)
        base_name = file.rsplit('_', 1)[0]  # Get the part before _FB1, _FB2, etc.
        grouped_files[base_name].append(file)

# List to hold paths of all created Excel files
excel_file_paths = []

# Process each group of files
for base_name, files in grouped_files.items():
    # Ensure there are exactly 4 files for this base name
    if len(files) == 4:
        # Sort files to ensure FB1, FB2, FB3, FB4 order
        files.sort(key=lambda x: int(x[-5]))

        # Dictionary to hold the CSV data
        csv_files = {}
        
        # Load the CSV files into the dictionary
        for file in files:
            suffix = file.rsplit('_', 1)[1].split('.')[0]  # Get FB1, FB2, etc.
            full_path = os.path.join(file_path, file)
            csv_files[suffix] = pd.read_csv(full_path)

        # Create an Excel writer object to save all the sheets into one file
        output_excel_path = os.path.join(file_path, f'{base_name}{excel_suffix}.xlsx')
        with pd.ExcelWriter(output_excel_path) as writer:
            for sheet_name, df in csv_files.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"CSV files for {base_name} successfully saved to {base_name}{excel_suffix}.xlsx.")
    else:
        print(f"Warning: {base_name} does not have exactly 4 corresponding CSV files.")


Combine excel files (Background Is also handeled)

In [ ]:
"""
STEP 2: Extract Rows from Channel Sheets and Perform Background Subtraction
---------------------------------------------------------------------------------

This script processes multiple Excel files generated from per-channel calcium imaging data.
It extracts intensity values from each file, combines them into a single DataFrame per channel,
and performs automated background subtraction.

Functionality:
- Scans a folder for all "<base_name>_analysis.xlsx" files.
- Groups files by base name (e.g., different repeats of the same sample).
- Extracts the second row of each channel sheet as the data line.
- Aligns data across up to 50 'Mean' values and appends 'Background'.
- Automatically determines background from the last valid Mean value.
- Subtracts this background to create corrected 'BS' (background-subtracted) columns.
- Prevents redundancy by setting the column used as background to NaN.

Assumptions:
- Excel sheets are named "Intensity FP1" to "Intensity FP4".
- The second row of each sheet contains the relevant signal values.
- Columns are structured as: [File name, Index, , Label, Mean1 to Mean50].

Outputs:
- One combined Excel file per base name: "<base_name>combined_analysis.xlsx"
  - Each sheet contains raw and background-subtracted data.
  - Columns: File info + Mean1–Mean50 + Background + BS1–BS50

Usage:
- Update `results_folder` to point to your Excel input directory.
- Run in a Python environment with `pandas` and `numpy` installed.
"""


import os
import pandas as pd
import numpy as np

# Folder containing the Excel files
results_folder = r"your\file\path"  # Replace with the path to your folder  

# Define the target headers for the combined file
headers = ['File name', 'Index', ' ', 'Label'] + [f'Mean{i}' for i in range(1, 51)] + ['Background']

# Loop through the files in the folder to identify unique base names
base_name_files = {}
for file_name in os.listdir(results_folder):
    if file_name.endswith(".xlsx") and '_analysis' in file_name:
        # Extract the base name (everything before the numbering and analysis suffix)
        base_name = file_name.rsplit('_', 2)[0] + '_'  # Improved base name extraction
        if base_name not in base_name_files:
            base_name_files[base_name] = []
        base_name_files[base_name].append(file_name)

# Process each set of files with the same base name
for base_name, file_names in base_name_files.items():
    # Dictionary to hold data for each sheet
    combined_data = {
        'Intensity FP1': [],
        'Intensity FP2': [],
        'Intensity FP3': [],
        'Intensity FP4': []
    }

    # Loop through the files with the same base name
    for file_name in file_names:
        # Load the Excel file
        file_path = os.path.join(results_folder, file_name)
        try:
            xls = pd.ExcelFile(file_path)
        except (ValueError, pd.errors.ExcelFileError) as e:
            print(f"Skipping file {file_name} due to error: {e}")
            continue

        # Loop through each sheet and extract the second row
        for sheet_name in combined_data.keys():
            if sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
                if len(df) > 1:
                    second_row = df.iloc[1].tolist()  # Extract the second row

                    # Ensure the second row has the correct length by padding or trimming
                    required_length = len(headers) - 2  # Subtracting 2 to account for 'File name' and 'Index'
                    if len(second_row) < required_length:
                        second_row.extend([np.nan] * (required_length - len(second_row)))
                    elif len(second_row) > required_length:
                        second_row = second_row[:required_length]

                    # Add the full file name and an index in front of the row
                    combined_row = [file_name, len(combined_data[sheet_name]) + 1] + second_row
                    combined_data[sheet_name].append(combined_row)

    # Create a combined file for the current base name
    output_file = os.path.join(results_folder, f'{base_name}combined_analysis.xlsx')
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Write each sheet's combined data to the output file
        for sheet_name, data in combined_data.items():
            combined_df = pd.DataFrame(data, columns=headers)

            # Identify numerical columns (assuming 'Mean1' to 'Mean50' are numerical)
            mean_cols = [f'Mean{i}' for i in range(1, 51)]
            bs_cols = [f'BS{i}' for i in range(1, 51)]

            # Implement the vectorized approach to find the last numerical value
            # Reverse the order of numerical columns
            reversed_numerical = combined_df[mean_cols].iloc[:, ::-1]

            # Perform backward fill along the columns
            filled = reversed_numerical.bfill(axis=1)

            # Select the first non-NaN value from the reversed columns, which corresponds to the last numerical value in original order
            combined_df['Background'] = filled.iloc[:, 0]

            # Step 1: Identify the 'MeanX' column used as 'Background' for each row
            # This is done by finding the first non-NaN 'Mean' column in the reversed order
            combined_df['Background_col'] = combined_df[mean_cols].apply(
                lambda row: row[::-1].first_valid_index(), axis=1
            )

            # Step 2: Subtract 'Background' from all 'Mean' columns to create 'BS' columns
            combined_df[bs_cols] = combined_df[mean_cols].subtract(combined_df['Background'], axis=0)

            # Step 3: For each row, set the 'BS' column corresponding to 'Background_col' to NaN
            # This removes the redundant zero
            for i in range(1, 51):
                bs_col = f'BS{i}'
                mean_col = f'Mean{i}'
                combined_df.loc[combined_df['Background_col'] == mean_col, bs_col] = np.nan

            # Step 4: (Optional) Drop the 'Background_col' if not needed
            combined_df.drop(columns=['Background_col'], inplace=True)

            # Optional: Reorder columns to place BS columns after Background
            # Define the new column order
            new_order = headers + bs_cols
            combined_df = combined_df[new_order]

            # Write the DataFrame to the Excel sheet
            combined_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Combined file saved as {output_file}")
